In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder \
    .appName("FlightDelay_Full_Models_LR_DT") \
    .getOrCreate()

In [3]:
fact = spark.read.csv("../output/CleanFactFlight.csv", header=True, inferSchema=True)

dim_time = spark.read.csv("../output/Dim_Time.csv", header=True, inferSchema=True)
dim_date = spark.read.csv("../output/Dim_Date.csv", header=True, inferSchema=True)
dim_airport = spark.read.csv("../output/Dim_Airport.csv", header=True, inferSchema=True)
dim_marketing = spark.read.csv("../output/Dim_Marketing_Airline.csv", header=True, inferSchema=True)
dim_operating = spark.read.csv("../output/Dim_Operating_Airline.csv", header=True, inferSchema=True)

In [4]:
df = fact.select(
    "flight_id",
    "date_id",
    "departure_time_id",
    "origin_airport_id",
    "destination_airport_id",
    "marketing_airline_id",
    "operating_airline_id",
    "dep_delay_minutes",
    "crs_elapsed_time",
    "distance",
    "flight_complexity_score"
)

In [5]:
df = df.filter(
    (col("dep_delay_minutes").isNotNull()) &
    (col("dep_delay_minutes") > -60) &
    (col("dep_delay_minutes") < 300)
)


In [6]:
df = df.join(
    dim_date.select(
        "date_id",
        "year",
        "quarter",
        "month",
        "day_of_month",
        "week_of_year",
        "is_weekend"
    ),
    on="date_id",
    how="left"
)

df = df.withColumn(
    "is_weekend_int",
    when((col("is_weekend") == True) | (col("is_weekend") == "true"), 1).otherwise(0)
)

In [7]:
df = df.join(
    dim_time.select(
        col("time_id").alias("dep_tid"),
        col("hour").alias("dep_hour"),
        col("minute").alias("dep_minute"),
        col("period_of_day").alias("dep_period_of_day")
    ),
    df.departure_time_id == col("dep_tid"),
    "left"
)

In [8]:
# origin airport
df = df.join(
    dim_airport.select(
        col("airport_id").alias("origin_id"),
        col("airport_code").alias("origin_airport_code"),
        col("state_name").alias("origin_state"),
        col("country").alias("origin_country")
    ),
    df.origin_airport_id == col("origin_id"),
    "left"
)

# destination airport
df = df.join(
    dim_airport.select(
        col("airport_id").alias("dest_id"),
        col("airport_code").alias("dest_airport_code"),
        col("state_name").alias("dest_state"),
        col("country").alias("dest_country")
    ),
    df.destination_airport_id == col("dest_id"),
    "left"
)

In [9]:
df = df.join(
    dim_marketing.select(
        col("marketing_airline_id").alias("mkt_id"),
        col("iata_code").alias("marketing_iata")
    ),
    df.marketing_airline_id == col("mkt_id"),
    "left"
)

df = df.join(
    dim_operating.select(
        col("operating_airline_id").alias("op_id"),
        col("iata_code").alias("operating_iata")
    ),
    df.operating_airline_id == col("op_id"),
    "left"
)

In [10]:
df = df.withColumn("delay_flag", when(col("dep_delay_minutes") > 30, 1).otherwise(0))

In [11]:
features_df = df.select(
    # classification target
    "delay_flag",

    # regression target
    "dep_delay_minutes",

    # numeric
    "crs_elapsed_time",
    "distance",
    "flight_complexity_score",
    "year",
    "quarter",
    "month",
    "day_of_month",
    "week_of_year",
    "is_weekend_int",
    "dep_hour",
    "dep_minute",

    # categorical
    "dep_period_of_day",
    "origin_airport_code",
    "origin_state",
    "origin_country",
    "dest_airport_code",
    "dest_state",
    "dest_country",
    "marketing_iata",
    "operating_iata"
).na.drop()

In [12]:
train, val, test = features_df.randomSplit([0.7, 0.15, 0.15], seed=42)

In [13]:
cat_cols = [
    "dep_period_of_day",
    "origin_airport_code",
    "origin_state",
    "origin_country",
    "dest_airport_code",
    "dest_state",
    "dest_country",
    "marketing_iata",
    "operating_iata"
]

num_cols = [
    "crs_elapsed_time",
    "distance",
    "flight_complexity_score",
    "year",
    "quarter",
    "month",
    "day_of_month",
    "week_of_year",
    "is_weekend_int",
    "dep_hour",
    "dep_minute"
]

indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in cat_cols]

encoder = OneHotEncoder(
    inputCols=[f"{c}_idx" for c in cat_cols],
    outputCols=[f"{c}_ohe" for c in cat_cols]
)

assembler = VectorAssembler(
    inputCols=num_cols + [f"{c}_ohe" for c in cat_cols],
    outputCol="features"
)

In [17]:
# Logistic Regression classification
lr = LogisticRegression(
    labelCol="delay_flag",
    featuresCol="features",
    maxIter=20,
    regParam=0.0
)

pipeline_lr = Pipeline(stages=indexers + [encoder, assembler, lr])
model_lr = pipeline_lr.fit(train)

val_pred_lr = model_lr.transform(val)
test_pred_lr = model_lr.transform(test)

clf_eval = MulticlassClassificationEvaluator(
    labelCol="delay_flag",
    predictionCol="prediction",
    metricName="accuracy"
)

print("LOGISTIC REGRESSION CLASSIFICATION")
print("Validation Accuracy:", clf_eval.evaluate(val_pred_lr))
print("Test Accuracy:", clf_eval.evaluate(test_pred_lr))

LOGISTIC REGRESSION CLASSIFICATION
Validation Accuracy: 0.9797421731123389
Test Accuracy: 0.980980751604033


In [15]:
# Decision Tree regression
dtr = DecisionTreeRegressor(
    labelCol="dep_delay_minutes",
    featuresCol="features",
    maxDepth=7
)

pipeline_dtr = Pipeline(stages=indexers + [encoder, assembler, dtr])
model_dtr = pipeline_dtr.fit(train)

val_pred_dtr = model_dtr.transform(val)
test_pred_dtr = model_dtr.transform(test)

eval_rmse = RegressionEvaluator(labelCol="dep_delay_minutes", predictionCol="prediction", metricName="rmse")
eval_mae  = RegressionEvaluator(labelCol="dep_delay_minutes", predictionCol="prediction", metricName="mae")
eval_mse  = RegressionEvaluator(labelCol="dep_delay_minutes", predictionCol="prediction", metricName="mse")
eval_r2   = RegressionEvaluator(labelCol="dep_delay_minutes", predictionCol="prediction", metricName="r2")

print("DECISION TREE REGRESSION METRICS - VALIDATION")
print("RMSE:", eval_rmse.evaluate(val_pred_dtr))
print("MAE :", eval_mae.evaluate(val_pred_dtr))
print("MSE :", eval_mse.evaluate(val_pred_dtr))
print("R2  :", eval_r2.evaluate(val_pred_dtr))
print()
print("DECISION TREE REGRESSION METRICS - TEST")
print("RMSE:", eval_rmse.evaluate(test_pred_dtr))
print("MAE :", eval_mae.evaluate(test_pred_dtr))
print("MSE :", eval_mse.evaluate(test_pred_dtr))
print("R2  :", eval_r2.evaluate(test_pred_dtr))


DECISION TREE REGRESSION METRICS - VALIDATION
RMSE: 8.968243794488087
MAE : 3.0943211355060583
MSE : 80.4293967573741
R2  : 0.9299863887949394

DECISION TREE REGRESSION METRICS - TEST
RMSE: 8.99708625757986
MAE : 3.0801502195394974
MSE : 80.94756112633236
R2  : 0.9289441533273689
